In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
import tqdm as nootbook_tqdm
from sklearn.ensemble import AdaBoostRegressor


c:\Users\sunrise\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv("sunny_cluster.csv")
df.head()

,GHI,Clearsky DHI,cluster,DHI,DNI,Wind Speed,Relative Humidity,Temperature,Clearsky DNI,Clearsky GHI,Cloud Type,Solar Zenith Angle
0,427,50,0,60,894,4.7,54.23,3.0,959,444,0,65.75
1,394,48,0,48,933,4.3,55.89,2.3,938,396,0,68.20
2,370,48,0,54,882,2.9,50.80,-0.9,947,387,0,69.01
3,438,53,0,55,956,2.9,48.99,-0.4,969,442,0,66.36
4,484,60,0,60,980,2.9,47.64,0.1,980,484,0,64.37


In [3]:
df.shape

(4885, 12)

In [4]:
df.isnull().sum()

GHI                   0
Clearsky DHI          0
cluster               0
DHI                   0
DNI                   0
Wind Speed            0
Relative Humidity     0
Temperature           0
Clearsky DNI          0
Clearsky GHI          0
Cloud Type            0
Solar Zenith Angle    0
dtype: int64

In [5]:
df.drop(['cluster'], axis = 1, inplace = True) 

In [6]:
df.describe()

,GHI,Clearsky DHI,DHI,DNI,Wind Speed,Relative Humidity,Temperature,Clearsky DNI,Clearsky GHI,Cloud Type,Solar Zenith Angle
count,4885.000000,4885.000000,4885.000000,4885.000000,4885.000000,4885.000000,4885.000000,4885.000000,4885.000000,4885.000000,4885.000000
mean,616.202456,91.901331,156.200614,726.331832,2.717994,27.690878,16.832487,919.222723,693.633982,1.891709,47.676669
std,192.619786,40.769207,110.079159,284.966113,1.321444,13.683449,10.539028,102.613445,198.179709,2.796603,14.640440
min,310.000000,37.000000,37.000000,23.000000,0.200000,5.980000,-12.600000,286.000000,321.000000,0.000000,16.250000
25%,455.000000,68.000000,74.000000,522.000000,1.800000,16.590000,8.300000,879.000000,522.000000,0.000000,36.120000
50%,588.000000,84.000000,105.000000,837.000000,2.600000,25.570000,18.400000,945.000000,688.000000,0.000000,49.730000
75%,763.000000,102.000000,221.000000,960.000000,3.400000,36.500000,26.000000,987.000000,864.000000,4.000000,60.360000
max,1060.000000,377.000000,512.000000,1092.000000,8.700000,77.970000,34.100000,1092.000000,1065.000000,9.000000,72.260000


In [7]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)

In [8]:
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['GHI']])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42,shuffle=False)

In [10]:
model = AdaBoostRegressor(n_estimators=100,learning_rate=0.1)
model.fit(X_train, y_train)
print(model)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


AdaBoostRegressor(learning_rate=0.1, n_estimators=100)


In [11]:
import time
start=time.time()
prediction = model.predict(X_test)
end=time.time()
continental=end-start
prediction = model.predict(X_test)
print("Inference time:",continental)

Inference time: 0.039998531341552734


In [12]:
prediction.shape

(977,)

In [13]:
prediction = prediction.reshape(prediction.shape[0], 1)

In [14]:
prediction.shape

(977, 1)

In [15]:
Pred= s2.inverse_transform(prediction)

In [16]:
Actual= s2.inverse_transform(y_test)

In [17]:
Calculated = pd.DataFrame(Pred, columns = ['Prediction'])

In [18]:
Calculated.to_csv(r'E:\CatBoost_1\Golden\AdaBoost\sunny hours\prediction.csv', index = False)

In [19]:
Actual = pd.DataFrame(Actual, columns = ['Actual'])

In [20]:
Actual.to_csv(r'E:\CatBoost_1\Golden\AdaBoost\sunny hours\Actual.csv', index = False)

In [21]:
x=Pred
z=Actual

In [22]:
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE


print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)


Root Mean Square Error: 17.477440121496183
Mean Square Error: 305.4609132004845
Mean Absolute Error: 15.588841619139162


In [23]:
max= x.max()
max

817.5384615384609

In [24]:
min=x.min()
min

369.5711711711711

In [25]:
NRMSE= (RMSE/(max-min))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 3.9014991713270795
